In [21]:
import io
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import time


# Download do dump:
!wget --quiet -P /tmp/ibd/ https://raw.githubusercontent.com/FranciscoNevesT/TrabalhoBD/main/trabalho.sql

In [22]:
conn = sqlite3.connect('/tmp/ibd/conn_trabalho.sql')
cursor = conn.cursor()

In [23]:
#Executar só uma vez a cada vez que abrir o colab
try:
  f = io.open('/tmp/ibd/trabalho.sql', 'r', encoding='ISO-8859-1')
  sql = f.read()
  cursor.executescript(sql)
except:
  print("falha")

falha


Modelo relacional
![picture](https://drive.google.com/uc?export=view&id=1IUlh5xQ4vYlVFNlvm7izcrpjvwrIPJQ5)

Liste o NU MATRICULA, Cidade, Uf e curso dos discentes que nasceram em CARUARU (consultas	envolvendo	a	junção	de	três	ou	mais	relações)

In [24]:
tempos1 = []
tempos2 = []

In [25]:
tempo = time.time()

query = """
Select D.NU_MATRICULA, Cidade, UF, Curso
From Discente as D, Cursa as C, Curso as Cr, Localidade as L
Where D.NU_MATRICULA = C.NU_MATRICULA AND C.CD_INEP = Cr.CD_INEP AND D.COD_LOCAL = L.COD_LOCAL AND Cidade = "CARUARU"

"""
df = pd.read_sql_query(query, conn)

tempos1.append(time.time() - tempo)
print(df)


      NU_MATRICULA   CIDADE  UF                         CURSO
0        200665077  CARUARU  PE            ADMINISTRAÇÃO -CAA
1        200715447  CARUARU  PE  ENGENHARIA DE PRODUÇÃO - CAA
2        200739520  CARUARU  PE                  DESIGN - CAA
3        200739949  CARUARU  PE  ENGENHARIA DE PRODUÇÃO - CAA
4        200741402  CARUARU  PE  ENGENHARIA DE PRODUÇÃO - CAA
...            ...      ...  ..                           ...
1453     200986234  CARUARU  PE        ENGENHARIA CIVIL - CAA
1454     200986245  CARUARU  PE                MEDICINA - CAA
1455     200986678  CARUARU  PE                        TEATRO
1456     200987143  CARUARU  PE      COMUNICAÇÃO SOCIAL - CAA
1457     200987234  CARUARU  PE      COMUNICAÇÃO SOCIAL - CAA

[1458 rows x 4 columns]


In [26]:
tempo = time.time()

query = """
Select NU_MATRICULA, Cidade, UF, Curso
From Curso Natural Join (Select NU_MATRICULA, CIDADE, UF, CD_INEP
                          From Cursa Natural join (Select NU_MATRICULA, CIDADE, UF
                                                  From Discente Natural Join (Select *
                                                                              From Localidade
                                                                              Where Cidade = "CARUARU")))

"""
df = pd.read_sql_query(query, conn)

tempos2.append(time.time() - tempo)
print(df)

      NU_MATRICULA   CIDADE  UF                         CURSO
0        200665077  CARUARU  PE            ADMINISTRAÇÃO -CAA
1        200715447  CARUARU  PE  ENGENHARIA DE PRODUÇÃO - CAA
2        200739520  CARUARU  PE                  DESIGN - CAA
3        200739949  CARUARU  PE  ENGENHARIA DE PRODUÇÃO - CAA
4        200741402  CARUARU  PE  ENGENHARIA DE PRODUÇÃO - CAA
...            ...      ...  ..                           ...
1453     200986234  CARUARU  PE        ENGENHARIA CIVIL - CAA
1454     200986245  CARUARU  PE                MEDICINA - CAA
1455     200986678  CARUARU  PE                        TEATRO
1456     200987143  CARUARU  PE      COMUNICAÇÃO SOCIAL - CAA
1457     200987234  CARUARU  PE      COMUNICAÇÃO SOCIAL - CAA

[1458 rows x 4 columns]


Liste o NU MATRICULA, o nome da deficiencia e o curso dos discentes que possuem alguma defieciencia fisica. (consultas	envolvendo	a	junção	de	três	ou	mais	relações)

In [27]:
tempo = time.time()

query = """
Select D.NU_MATRICULA, NOME, CURSO
From Discente as D, Possui P, DEFICIENCIA as De, Cursa as C, Curso as Cr
Where D.NU_MATRICULA = C.NU_MATRICULA AND C.CD_INEP = Cr.CD_INEP AND D.NU_MATRICULA = P.NU_MATRICULA AND P.COD_DEFICIENCIA = De.COD_DEFICIENCIA AND Tipo = "FISICA"

"""
df = pd.read_sql_query(query, conn)

tempos1.append(time.time() - tempo)
print(df)

     NU_MATRICULA      NOME                            CURSO
0       200711907    FISICA          ARQUITETURA E URBANISMO
1       200711973  MULTIPLA                EXPRESSÃO GRÁFICA
2       200780001    FISICA  LETRAS PORTUGUÊS - LICENCIATURA
3       200802890    FISICA                      ODONTOLOGIA
4       200803658    FISICA                       MUSEOLOGIA
..            ...       ...                              ...
269     200986665  AUDITIVA                         MEDICINA
270     200986707    FISICA                         MEDICINA
271     200986722  AUDITIVA                        PEDAGOGIA
272     200987084  AUDITIVA                         ISOLADAS
273     200987196    SURDEZ                         MEDICINA

[274 rows x 3 columns]


In [28]:
tempo = time.time()

query = """
Select NU_MATRICULA, NOME, CURSO

From Cursa Natural Join  (Select NU_MATRICULA, Nome
      From Discente Natural Join Possui Natural Join Deficiencia 
      Where TIPO == "FISICA") Natural Join Curso
"""
df = pd.read_sql_query(query, conn)

tempos2.append(time.time() - tempo)
print(df)

     NU_MATRICULA      Nome                            CURSO
0       200711907    FISICA          ARQUITETURA E URBANISMO
1       200711973  MULTIPLA                EXPRESSÃO GRÁFICA
2       200780001    FISICA  LETRAS PORTUGUÊS - LICENCIATURA
3       200802890    FISICA                      ODONTOLOGIA
4       200803658    FISICA                       MUSEOLOGIA
..            ...       ...                              ...
269     200986665  AUDITIVA                         MEDICINA
270     200986707    FISICA                         MEDICINA
271     200986722  AUDITIVA                        PEDAGOGIA
272     200987084  AUDITIVA                         ISOLADAS
273     200987196    SURDEZ                         MEDICINA

[274 rows x 3 columns]


Liste o NU_MATRICULA, cidade e o curso dos discentes que nasceram na cidade em que nasceram o maior numero de discentes (consultas	envolvendo	funções	de	agregação	sobre	o	resultado	da	junção	de	pelo	menos	duas	relações)

In [29]:
tempo = time.time()

query = """
Select D.NU_MATRICULA, Cidade, Curso
From Discente as D, Cursa as C, Curso as Cr,  (Select L.COD_LOCAL,Cidade,Count(NU_MATRICULA) as quantidade
                                                From Localidade as L, Discente as D
                                                Where D.COD_LOCAL = L.COD_LOCAL
                                                Group BY CIDADE
                                                Order by quantidade desc
                                                Limit 1) as V
Where D.COD_LOCAL = V.COD_LOCAL AND D.NU_MATRICULA = C.NU_MATRICULA AND C.CD_INEP = Cr.CD_INEP

"""
df = pd.read_sql_query(query, conn)

tempos1.append(time.time() - tempo)
print(df)

       NU_MATRICULA  Cidade                        CURSO
0         200102627  RECIFE                     GEOLOGIA
1         200306046  RECIFE                     ISOLADAS
2         200306372  RECIFE                     ISOLADAS
3         200406008  RECIFE                     ISOLADAS
4         200644864  RECIFE      GEOGRAFIA - BACHARELADO
...             ...     ...                          ...
17554     200987197  RECIFE                      DIREITO
17555     200987226  RECIFE  CIÊNCIA DA COMPUTAÇÃO - CIN
17556     200987228  RECIFE             ABI - ENGENHARIA
17557     200987235  RECIFE                   JORNALISMO
17558     200987244  RECIFE          CIÊNCIAS ECONÔMICAS

[17559 rows x 3 columns]


In [30]:
tempo = time.time()

query = """
Select NU_MATRICULA, Cidade, Curso
From Curso Natural Join Cursa Natural Join (Select NU_MATRICULA, Cidade
                                            From Discente Natural Join (Select COD_LOCAL,Cidade,Count(NU_MATRICULA) as quantidade
                                                                             From Localidade Natural Join Discente as D
                                                                             Group BY CIDADE
                                                                             Order by quantidade desc
                                                                             Limit 1) )



"""
df = pd.read_sql_query(query, conn)

tempos2.append(time.time() - tempo)
print(df)

       NU_MATRICULA  Cidade                        CURSO
0         200102627  RECIFE                     GEOLOGIA
1         200306046  RECIFE                     ISOLADAS
2         200306372  RECIFE                     ISOLADAS
3         200406008  RECIFE                     ISOLADAS
4         200644864  RECIFE      GEOGRAFIA - BACHARELADO
...             ...     ...                          ...
17554     200987197  RECIFE                      DIREITO
17555     200987226  RECIFE  CIÊNCIA DA COMPUTAÇÃO - CIN
17556     200987228  RECIFE             ABI - ENGENHARIA
17557     200987235  RECIFE                   JORNALISMO
17558     200987244  RECIFE          CIÊNCIAS ECONÔMICAS

[17559 rows x 3 columns]


Liste o NU_MATRICULA, cidade e o nome do curso dos discentes que cursam o curso com o maior numero de discentes (consultas	envolvendo	funções	de	agregação	sobre	o	resultado	da	junção	de	pelo	menos	duas	relações)

In [31]:
tempo = time.time()

query = """
Select D.NU_MATRICULA, Cidade, Curso
From Discente as D,  Cursa as C, Localidade as L, (Select C.CD_INEP, CURSO, Count(D.NU_MATRICULA) as quantidade
                                                  From Discente as D, Cursa as C, Curso as Cr
                                                  Where D.NU_MATRICULA = C.NU_MATRICULA AND C.CD_INEP = Cr.CD_INEP
                                                  Group By C.CD_INEP
                                                  Order by quantidade desc
                                                  Limit 1) as V
Where C.CD_INEP = V.CD_INEP AND D.NU_MATRICULA = C.NU_MATRICULA AND L.COD_LOCAL = D.COD_LOCAL


"""
df = pd.read_sql_query(query, conn)

tempos1.append(time.time() - tempo)
print(df)

      NU_MATRICULA     CIDADE    CURSO
0        200710348     RECIFE  DIREITO
1        200740648     RECIFE  DIREITO
2        200741988     OLINDA  DIREITO
3        200744576     OLINDA  DIREITO
4        200770912     RECIFE  DIREITO
...            ...        ...      ...
1315     200985522     RECIFE  DIREITO
1316     200986460  GUARABIRA  DIREITO
1317     200986641     TABIRA  DIREITO
1318     200987197     RECIFE  DIREITO
1319     200987229  ARCOVERDE  DIREITO

[1320 rows x 3 columns]


In [32]:
tempo = time.time()

query = """
Select NU_MATRICULA, Cidade, Curso
From Localidade Natural Join (Select NU_MATRICULA,CURSO,COD_LOCAL
                              From Discente Natural Join Cursa Natural Join (Select CD_INEP, Count(NU_MATRICULA) as numero
                                                                              From Discente Natural Join Cursa 
                                                                              Group By CD_INEP
                                                                              Order By numero desc
                                                                              Limit 1) Natural Join Curso)



"""
df = pd.read_sql_query(query, conn)

tempos2.append(time.time() - tempo)
print(df)

      NU_MATRICULA     CIDADE    CURSO
0        200710348     RECIFE  DIREITO
1        200740648     RECIFE  DIREITO
2        200741988     OLINDA  DIREITO
3        200744576     OLINDA  DIREITO
4        200770912     RECIFE  DIREITO
...            ...        ...      ...
1315     200985522     RECIFE  DIREITO
1316     200986460  GUARABIRA  DIREITO
1317     200986641     TABIRA  DIREITO
1318     200987197     RECIFE  DIREITO
1319     200987229  ARCOVERDE  DIREITO

[1320 rows x 3 columns]


In [33]:
for x in range(len(tempos1)):
  print("{}: {} | {}".format(x,tempos1[x],tempos2[x]))

0: 0.03570985794067383 | 0.029215097427368164
1: 0.020254135131835938 | 0.019949674606323242
2: 0.07918858528137207 | 0.07682490348815918
3: 0.06134295463562012 | 0.032758235931396484
